In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u
import pandas as pd

import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
cptac warning: Your version of cptac (0.8.0) is out-of-date. Latest is 0.8.1. Please run 'pip install --upgrade cptac' to update it. (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 10)


In [21]:
def wrap_ttest(df, label_column, comparison_columns=None, alpha=.05, return_all=False, correction_method='bonferroni', mincount=3):
    try:
        '''Verify precondition that label column exists and has exactly 2 unique values'''
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        '''Partition dataframe into two sets, one for each of the two unique values from the label column'''
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]

        '''If no comparison columns specified, use all columns except the specified labed column'''
        if not comparison_columns:
            comparison_columns = list(df.columns)
            comparison_columns.remove(label_column)
 
        '''Determine the number of real valued columns on which we will do t-tests'''
        number_of_comparisons = len(comparison_columns)

        '''Store comparisons and p-values in two arrays'''
        comparisons = []
        pvals = []
        
        '''Loop through each comparison column, perform the t-test, and record the p-val'''
        for column in comparison_columns:
            if len(partition1[column].dropna(axis=0)) <= mincount:
        #         comparison_columns.remove(column)
                continue
            elif len(partition2[column].dropna(axis=0)) <= mincount:
        #         comparison_columns.remove(column)
                continue
            else:
                stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
                comparisons.append(column)
                pvals.append(pval)
            
        '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
        results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
        reject = results[0]

        '''Format results in a pandas dataframe'''
        results_df = pd.DataFrame(columns=['Comparison','P_Value'])

        '''If return all, add all comparisons and p-values to dataframe'''
        if return_all:
            results_df['Comparison'] = comparisons
            results_df['P_Value'] = pvals

            '''Else only add significant comparisons'''
        else:
            for i in range(0, len(reject)):
                if reject[i]:
                    results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


        '''Sort dataframe by ascending p-value'''
        results_df = results_df.sort_values(by='P_Value', ascending=True)
        results_df = results_df.reset_index(drop=True)

        '''If results df is not empty, return it, else return None'''
        if len(results_df) > 0:
            return results_df
        else:
            print("Empty")
            return results[1]


    except:
        print("Incorrectly Formatted Dataframe!")
        return None

# Load cancer data

In [3]:
en = cptac.Endometrial()
br = cptac.Brca()
cl = cptac.Colon()

In [20]:
mut = en.get_genotype_all_vars("PIK3CA")
# mut

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


In [19]:
prot = en.get_proteomics(tissue_type="tumor")
joined = mut.join(prot)
# joined.head()

In [18]:
prot_and_mutation = joined.loc[(joined["Mutation"] == "Missense_Mutation") | (joined["Mutation"] == "Wildtype_Tumor")]

prot_and_mutations = prot_and_mutation.drop(columns = ["Mutation_Status", "Location"])
# prot_and_mutations.head(40)

In [12]:
len(prot_and_mutation.loc[prot_and_mutation["Mutation"]=="Missense_Mutation"])
len(prot_and_mutation.loc[prot_and_mutation["Mutation"]=="Wildtype_Tumor"])


# len(mut.index)

41

In [13]:
partition1 = prot_and_mutations.loc[prot_and_mutations["Mutation"] == "Missense_Mutation"]
partition2 = prot_and_mutations.loc[prot_and_mutations["Mutation"] == "Wildtype_Tumor"]

comparison_columns = list(prot_and_mutations.columns)
comparison_columns.remove("Mutation")

genelist = []
for column in comparison_columns:
    if(len(partition1[column].dropna(axis=0))) <= 40 and (len(partition2[column].dropna(axis=0))) <= 40:
        continue
    else:
        genelist.append(column)

In [14]:
#let's try filtering prot_and_mutations (so that we can look at the pvals)
# fil_prot = prot_and_mutations[[genelist]]
genelist.append("Mutation")
fil_prot = prot_and_mutations.loc[:, prot_and_mutations.columns.isin(genelist)]
len(prot_and_mutations.columns)
len(fil_prot.columns)

8343

In [ ]:
len(genelist)

In [23]:
# wrap_ttest(prot_and_mutations, 'Mutation', correction_method = 'fdr_bh', mincount=40)
n = wrap_ttest(fil_prot, 'Mutation', correction_method = 'fdr_bh', mincount=40, alpha=.5)
min(n)

Empty


0.9994782839432604

In [ ]:
#try just running the top 20
top20genes = list(n.head(20)["Comparison"])
top20genes.append("Mutation")
top20 = prot_and_mutations.loc[:, prot_and_mutations.columns.isin(top20genes)]
wrap_ttest(top20, 'Mutation', correction_method = 'fdr_bh', mincount=40)


#run in brca (there are probably more significant results)
#try looking for a different method, or raise the cutoff value

In [ ]:
prot_and_mutations["Mutation"].unique()

# Endometrial

## Filter out hotspot mutations
Hotspots are:
E542K
E545K
H1047R


In [ ]:
#### Get the mutation type, and ptorteomics for PIK3CA

en.get_genotype_all_vars(gene)

len(mut.loc[mut["Mutation"]=="Missense_Mutation"])

In [ ]:
prot = en.get_proteomics(tissue_type="tumor")

#### Join mutation type and proteomics together

In [ ]:
joined = mut.join(prot)
joined.head()

#### Select samples containing missense mutations

In [ ]:
# en_hotspot_df = joined[joined.Location.str.contains('E542K') | 
#                     joined.Location.str.contains('E545K') |
#                     joined.Location.str.contains('H1047R')]
missense_mutations = joined.loc[joined["Mutation"] == "Missense_Mutation"]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

#### Join the proteomics for the wildtype mutations to the hotspot mutations

In [ ]:
c# wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

# endo_hotspot = en_hotspot_df.append(wildtype)
endo_missense

#### Drop unncecesary columns
The resulting dataframe is what we will be working with from this point forward. It contains the mutations type and the proteomics for each gene.

In [ ]:
prot_and_mutations = endo_missense.drop(columns = ["Mutation_Status", "Location"])
prot_and_mutations.head()

## Run T-test and difference of Median for each cancer type

#### Get the median proteomic expression among all samples, for each gene.
This is done for missense mutations and wildtype separately


In [ ]:
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #get all wildtype mutations
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype

#### Get the medain difference between missense and wildtype for each gene.
This is done by (for each gene) subtracting the median of the missense mutations from the median of the wildtype.
This means that if the difference is positive the proteomics of the wildtype have higher expression than the missense mutation. If the difference is negative then the missense mutation has a higher expression than the wildtype.

In [ ]:
endo_d = {}

for gene in prot_and_mutations:
    if gene == "Mutation": continue
    dif_endo =  missense_med[gene] - wt_med[gene]
    endo_d[gene] = dif_endo

median_diff = pd.DataFrame.from_dict(endo_d, orient='index', columns=['Difference_In_Median'])
median_diff.head()

In [ ]:
len(prot_and_mutations.loc[prot_and_mutations['Mutation']=="Missense_Mutation"])

In [ ]:
prot_and_mutations

In [ ]:
u.wrap_ttest(prot_and_mutations, 'Mutation')



In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

In [ ]:
 prot_and_mutations["Mutation"].unique()
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u


In [ ]:
mincount = 30
label_values = prot_and_mutations["Mutation"].unique()

'''Partition dataframe into two sets, one for each of the two unique values from the label column'''
partition1 = prot_and_mutations.loc[prot_and_mutations["Mutation"] == label_values[0]]
partition2 = prot_and_mutations.loc[prot_and_mutations["Mutation"] == label_values[1]]

'''If no comparison columns specified, use all columns except the specified labed column'''
comparison_columns = list(prot_and_mutations.columns)
comparison_columns.remove("Mutation")

number_of_comparisons = len(comparison_columns)

'''Store comparisons and p-values in two arrays'''
comparisons = []
pvals = []

'''Loop through each comparison column, perform the t-test, and record the p-val'''
times_through = 0
for column in comparison_columns:  
    if len(partition1[column].dropna(axis=0)) <= mincount:
#         comparison_columns.remove(column)
        continue
    elif len(partition2[column].dropna(axis=0)) <= mincount:
#         comparison_columns.remove(column)
        continue
    else:
        stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
        comparisons.append(column)
        pvals.append(pval)
    
'''Correct for multiple testing to determine if each comparison meets the new cutoff'''
results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=.05, method='fdr_bh')
reject = results[0]


'''Format results in a pandas dataframe'''
results_df = pd.DataFrame(columns=['Comparison','P_Value'])

for i in range(0, len(reject)):
    if reject[i]:
        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


'''Sort dataframe by ascending p-value'''
results_df = results_df.sort_values(by='P_Value', ascending=True)
results_df = results_df.reset_index(drop=True)

In [ ]:
results_df

In [ ]:
len(pvals)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')
alt.Chart(pd.DataFrame({"p_val":pvals})).mark_bar().encode(
    x=alt.X("p_val:Q",bin=alt.Bin(step=0.05)),
    y="count()"
)

#### Do a t-test for every gene.

In [ ]:
genes = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation')
# t_test = t_test.set_index('Comparison')
t_test

#### Join difference in median and t-test p value into the same dataframe

In [ ]:
# Step 4 join median and p value together
endo_df = median_diff.join(t_test)
endo_df.head()

# Brca
The Brca data uses different database_IDs, giving us multiple proteomic values for each gene. To deal with this, we will first separate genes that have multiple proteomics values recoreded (multiple database_IDs) from ones that don't.

## Dealing with multiple database_IDs

#### Separate genes with multiple database_IDs from genes with multiple database_IDs

In [ ]:
proteomics = br.get_proteomics(tissue_type="tumor")
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
no_duplicates = proteomics[proteomics.columns[~fil]]
no_duplicates

### Dealing with genes that don't have multiple database_IDs
These will be processed the same way the data for endometrial was.

In [ ]:
#FOR NO DUPLICATES
#Flatten the multiindex (since there are no duplicate gene names, this won't be a problem)
no_duplicates = cptac.utils.reduce_multiindex(no_duplicates, flatten=True)

#get mutation_tyoe
mut_type = br.get_genotype_all_vars('PIK3CA')

#join proteomics and mutation type
joined = mut_type.join(no_duplicates)

missense_mutations = joined.loc[joined["Mutation"] == "Missense_Mutation"]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
brca_missense = missense_mutations.append(wildtype)


#drop unncessary columns
prot_and_mutations = brca_missense.drop(columns = ["Mutation_Status", "Location"])

# Get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


no_dup_d = {}

for prot in no_duplicates:
    dif_brca = missense_med[prot] - wt_med[prot]
    no_dup_d[prot] = dif_brca

median_diff = pd.DataFrame.from_dict(no_dup_d, orient='index', columns=['Difference_In_Median'])

#Do a t test for every gene and report the p-value
genes = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', correction_method="fdr_bh")
t_test = t_test.set_index('Comparison')

#join median and p value together
no_dup_df = median_diff.join(t_test)
no_dup_df.head()

## Dealing with genes that do have multiple database_IDs

#### Get t-test results and median difference

In [ ]:
#DEALING WITH DUPLICATES
#flatten the multiindex
duplicates = cptac.utils.reduce_multiindex(duplicates, flatten=True)

# We need to figure which of databaseID we want to use, for each protein
proteomics = br.get_proteomics(tissue_type="tumor")
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
duplicate_gene_names = list(set(duplicates.columns.get_level_values('Name'))) #get a list of genes names that are duplicated
duplicates = cptac.utils.reduce_multiindex(duplicates, flatten=True)
duplicates

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest

mut_type = br.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(duplicates)

missense_mutations = joined.loc[joined["Mutation"] == "Missense_Mutation"]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
brca_missense = missense_mutations.append(wildtype)


#drop unncessary columns
prot_and_mutations = brca_missense.drop(columns = ["Mutation_Status", "Location"])


# get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


duplicates_d = {}

for prot in duplicates:
    dif_brca =  missense_med[prot] - wt_med[prot]
    duplicates_d[prot] = dif_brca
    

median_diff = pd.DataFrame.from_dict(duplicates_d, orient='index', columns=['Difference_In_Median'])


#step 3 do a t test for every gene and report the p-value
cols = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', correction_method="fdr_bh")
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
dup_df = median_diff.join(t_test)
dup_df.head()

#### Select the database_ID that gives a lower p value from the t-test, and append selected database_ID to the no_duplicate dataframe

In [ ]:
#for each duplicated gene name select the one with the min p val and append to no_duplicat df
for name in duplicate_gene_names:
    gene_duplicate = dup_df[dup_df.index.str.contains(name)]
    #select the one with the min p value for t test
    min_row = gene_duplicate[gene_duplicate.P_Value == gene_duplicate.P_Value.min()]
    #append the selected version onto no_duplicate dataframe
    no_dup_df=no_dup_df.append(min_row)
no_dup_df=no_dup_df.sort_index()
no_dup_df.head()

#### Reformat the table by getting rid of databaseIDs so we can join it to Endo and Brca

In [ ]:
split = no_dup_df.index.str.split('_', 1).str[0]
no_dup_df['new_index']=split
brca_df = no_dup_df.set_index("new_index")
brca_df.head()

# Colon

In [ ]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = cl.get_proteomics(tissue_type="tumor")

prot_list = list(proteomics.columns)

mut_type = cl.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(proteomics)
joined["Mutation"] = joined['Mutation'].replace(['nonsynonymous SNV'], 'Missense_Mutation')


#select samples containing hotspot mutations
missense_mutations = joined.loc[joined["Mutation"] == "Missense_Mutation"]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
colon_missense = missense_mutations.append(wildtype)


#drop unncessary columns
prot_and_mutations = colon_missense.drop(columns = ["Mutation_Status", "Location"])


#get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "nonsynonymous SNV"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


colon_d = {}

for prot in proteomics:
    dif_colon = missense_med[prot] - wt_med[prot]
    colon_d[prot] = dif_colon

median_diff = pd.DataFrame.from_dict(colon_d, orient='index', columns=['Difference_In_Median'])

#do a t test for every gene and report the p-value
cols = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', correction_method="fdr_bh")
t_test = t_test.set_index('Comparison')

#join median and p value together
colon_df = median_diff.join(t_test)
colon_df.head()

# Combine Endo, Brca, and Colon into one table

#### Combine brca and endo

In [ ]:
brca_df=brca_df.rename_axis(None)
brca_endo = brca_df.join(endo_df, lsuffix='_Brca', rsuffix='_Endo')
brca_endo.head()

#### Combine colon

In [ ]:
final_table = brca_endo.join(colon_df, rsuffix='_colon')
final_table.head()

#### Only rows that contain a significant pvalue

In [ ]:
significant = final_table.loc[(final_table['P_Value_Brca'] <= .05) |
                (final_table['P_Value_Endo'] <= .05) |
                (final_table['P_Value'] <= .05)]
significant.head()

In [ ]:
significant.to_csv("~/WhenMutationsDontMatter/PIK3CA/csv_files/proteomics_no_hotspots.csv")